#Titanic - Machine Learning from a Disaster


In [ ]:
import pandas
import numpy
from sklearn import neighbors
from sklearn.neighbors import DistanceMetric 
from pprint import pprint

MY_TITANIC_TRAIN ='./sample_data/train.csv'
MY_TITANIC_TEST = './sample_data/test.csv'
titanic_dataframe = pandas.read_csv(MY_TITANIC_TRAIN)
print('length: {0} '.format(len(titanic_dataframe)))
titanic_dataframe.head(2)






In [ ]:
ave_age = 29.699118
titanic_dataframe['Age'].fillna(ave_age, inplace=True)





We replaced the NA age values with the mean ages to complete data set.

In [ ]:
titanic_dataframe = titanic_dataframe.dropna()

titanic_dataframe['Sex'].unique()
titanic_dataframe['Gender'] = titanic_dataframe['Sex'].map({'female': 0, 'male': 1}).astype(int)
titanic_dataframe['Embarked'].unique()
titanic_dataframe['Port'] = titanic_dataframe['Embarked'].map({'C':1, 'S':2, 'Q':3}).astype(int)

titanic_dataframe.drop(['Sex', 'Embarked', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [ ]:
cols = titanic_dataframe.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]
titanic_dataframe = titanic_dataframe[cols]

train_data = titanic_dataframe[cols[2: ]]
train_target = titanic_dataframe[cols[1]]

pprint('column_list: {0}'.format(cols))
print(train_data.head(2))

In [ ]:
titanic_dataframe.head(5)

In [ ]:
def hamming_distance(input_row, training_row):
    distance = list(
            # 1 if isinstance(input_row[i], str) & training_row[i] != input_row[i] else 0
            1 if input_row[key] != training_row[key] else 0 for key in factors
        # for i in range(len(input_row))
    )
    return sum(distance)

factors = [ 'Pclass', 'Port', 'Gender']
hamming = DistanceMetric.get_metric('hamming')

In [ ]:
for train_row in train:
    hamming_distance(train_row)

In [ ]:
def euclidian_distance(row, train_row):
    distance = list((row[key] - train_row[key])^2 for key in numericals)
                     # for i in range(len(input_row)) if isnumeric(row[i]) & isnumeric(train_row[i])
               # )
    return sqrt(sum(distance))

numericals = ['Age', 'SibSp', 'Parch', 'Fare']
euclidian = DistanceMetric.get_metric('euclidean')

In [ ]:
def knn(input_row, train, k):
    distance = []
    # for train_row in train:
    for row in train.iterrows():
        # that means row = tuple and i can get what i want by saying row[1]['Column Name']            
        hamming = hamming_distance(input_row[1], row[1])
        euclidian = euclidian_distance(input_row[1], row[1])
        distance.append(hamming + euclidian)
    distance.sort(key=operator.itemgetter(0))
    out = distance[ :k]
    return (input_row[1]['id'], 1 if sum(out) > k//2 else 0)

who_dies = []

#
    
for row in titanic_dataframe.iterrows():
   print(knn(row, titanic_dataframe, 10))
# knn(titanic_dataframe.iterrows(), titanic_dataframe, 10)